In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
from pavlov import stats, runs, storage, files
from boardlaw import hex
from boardlaw.main import mix
import torch
from tqdm.auto import tqdm

In [ ]:
m = storage.load_raw(run, 'model')
worlds = mix(hex.Hex.initial(n_envs=16*1024, boardsize=runs.info(run)['params']['boardsize']))

logits = {}
for idx in tqdm(storage.snapshots(run)):
    sd = storage.load_snapshot(run, idx)['agent']
    m.load_state_dict(storage.expand(sd)['network'])
    logits[idx] = m(worlds).logits.detach()
    
kldivs = {}
for i in logits:
    for j in logits:
        li = logits[i]
        lj = logits[j]
        terms = -li.exp().mul(lj - li)
        mask = torch.isfinite(terms)
        kldiv = terms.where(mask, torch.zeros_like(terms)).sum(-1)/mask.float().sum(-1)
        kldivs[i, j] = kldiv.mean().item()
df = pd.Series(kldivs).unstack()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, (t, b) = plt.subplots(2, 1, sharex=True)
t.plot(np.diag(df, 1))
t.grid(True)
t.set_title('kl div between 15-min interval snapshots')

stats.pandas('*gross-steams', 'elo-mohex', 'μ').ffill().rename(index=lambda t: t.total_seconds()/900).iloc[3:].plot(ax=b, grid=True)
b.set_title('elo-mohex')
b.set_xlabel('snapshot number')

fig.suptitle(runs.resolve(run))